In [22]:
#!/opt/anaconda/envs/bd9/bin/python

import time

#t0 = time.clock()

import sys
import pandas as pd
import numpy as np
import re
import json
import pickle
from urllib.parse import urlparse
from urllib.request import urlretrieve, unquote
from pandas.io.json import json_normalize
import datetime
from sklearn import preprocessing
import pickle


def url2domain(url):
    url = re.sub('(http(s)*://)+', 'http://', url)
    parsed_url = urlparse(unquote(url.strip()))
    if parsed_url.scheme not in ['http','https']: return None
    netloc = re.search("(?:www\.)?(.*)", parsed_url.netloc).group(1)
    if netloc is not None: return str(netloc.encode('utf8')).strip()
    return None

def datetime_conv(value):
    return datetime.datetime.fromtimestamp(value / 1e3)

In [23]:
#columns=['gender','age','uid','user_json']

#df = pd.read_table(sys.stdin, sep='\t', header=None, names=columns)

file_path = '/data/share/project01/gender_age_dataset.txt'
df = pd.read_csv(file_path, sep='\t')

df = df[((df.gender == '-') & (df.age == '-'))]
df.head()

,gender,age,uid,user_json
36138,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,"{""visits"": [{""url"": ""http://www.interfax.ru/bu..."
36139,-,-,bd7a6f52-45db-49bf-90f2-a3b07a9b7bcd,"{""visits"": [{""url"": ""https://www.packagetrackr..."
36140,-,-,bd7a7fd9-ab06-42f5-bf0f-1cbb0463004c,"{""visits"": [{""url"": ""http://www.mk.ru/incident..."
36141,-,-,bd7c5d7a-0def-41d1-895f-fdb96c56c2d4,"{""visits"": [{""url"": ""http://www.24open.ru/user..."
36142,-,-,bd7e54a2-0215-45cb-a869-9efebf250e38,"{""visits"": [{""url"": ""http://www.dns-shop.ru/ca..."


In [24]:
#df.drop(labels=['gender','age'],axis=1, inplace=True)

df.reset_index(drop=True, inplace=True)

df['user_json'] = json_normalize(df['user_json'].apply(json.loads))

df2_dict = df.to_dict('records')

In [25]:
df2_norm = pd.DataFrame(json_normalize(df2_dict, 'user_json', ['gender','age','uid']),columns=['gender','age','uid','timestamp','url'])
df2_norm.head()

,gender,age,uid,timestamp,url
0,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,1419775945781,http://www.interfax.ru/business/414668
1,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,1419679865088,http://amerikan-gruzovik.ru/zapchasti-dlya-ame...
2,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,1419679853405,http://amerikan-gruzovik.ru/
3,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,1419679847246,http://amerikan-gruzovik.ru/b-u-zapchasti-dlya...
4,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,1419679822992,http://amerikan-gruzovik.ru/


In [26]:
df2_norm['domain'] = df2_norm['url'].apply(url2domain)

df2_norm['datetime'] = df2_norm['timestamp'].apply(datetime_conv)
df2_norm['datetime'] = pd.to_datetime(df2_norm['datetime'], format='%Y%m%d%h%')
df3 = df2_norm.drop(labels=['url','timestamp'],axis=1)

In [27]:
df3.head()

,gender,age,uid,domain,datetime
0,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,b'interfax.ru',2014-12-28 17:12:25.781
1,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,b'amerikan-gruzovik.ru',2014-12-27 14:31:05.088
2,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,b'amerikan-gruzovik.ru',2014-12-27 14:30:53.405
3,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,b'amerikan-gruzovik.ru',2014-12-27 14:30:47.246
4,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,b'amerikan-gruzovik.ru',2014-12-27 14:30:22.992


In [28]:
df3['hour'] = df3['datetime'].dt.hour
df3['day'] = df3['datetime'].dt.day
df3['month'] = df3['datetime'].dt.month

df3['day_part'] = 'Unknown'
df3.loc[df3['hour'].between(6, 12, inclusive=True), 'day_part'] = 'Morning'
df3.loc[df3['hour'].between(13, 18, inclusive=True), 'day_part'] = 'Afternoon'
df3.loc[df3['hour'].between(19, 24, inclusive=True), 'day_part'] = 'Evening'
df3.loc[df3['hour'].between(0, 5, inclusive=True), 'day_part'] = 'Night'

df3['season'] = 'Unknown'
df3.loc[df3['month'].between(3, 5, inclusive=True), 'season'] = 'Spring'
df3.loc[df3['month'].between(6, 8, inclusive=True), 'season'] = 'Summer'
df3.loc[df3['month'].between(9, 11, inclusive=True), 'season'] = 'Autumn'
df3.loc[df3['month'].between(1, 2, inclusive=True), 'season'] = 'Winter'
df3.loc[df3['month'].between(12, 12, inclusive=True), 'season'] = 'Winter'

df3['week_day'] = df3['datetime'].dt.day_name()
workdays = ['Wednesday', 'Thursday', 'Friday', 'Monday', 'Tuesday']
weekend = ['Saturday', 'Sunday']

df3['day_type'] = df3['week_day'].isin(workdays)
df3['day_type'] = df3['day_type'].replace([True, False], ['Workday', 'Weekend'])

In [29]:
df3.head()

,gender,age,uid,domain,datetime,hour,day,month,day_part,season,week_day,day_type
0,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,b'interfax.ru',2014-12-28 17:12:25.781,17,28,12,Afternoon,Winter,Sunday,Weekend
1,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,b'amerikan-gruzovik.ru',2014-12-27 14:31:05.088,14,27,12,Afternoon,Winter,Saturday,Weekend
2,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,b'amerikan-gruzovik.ru',2014-12-27 14:30:53.405,14,27,12,Afternoon,Winter,Saturday,Weekend
3,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,b'amerikan-gruzovik.ru',2014-12-27 14:30:47.246,14,27,12,Afternoon,Winter,Saturday,Weekend
4,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,b'amerikan-gruzovik.ru',2014-12-27 14:30:22.992,14,27,12,Afternoon,Winter,Saturday,Weekend


In [123]:
df4 = df3.drop_duplicates(subset=['uid','domain','day_part', 'season', 'week_day', 'day_type'], keep='first')

df4.drop(labels=['datetime','hour','day','month'],axis=1, inplace=True)

df4.reset_index(drop=True, inplace=True)

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [158]:
df4_a = df4[df4["domain"].isin(list(set(sex_age_stat['domain']).intersection(set(df4['domain']))))]

In [159]:
df4_a.head()

,gender,age,uid,domain,day_part,season,week_day,day_type
0,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,b'interfax.ru',Afternoon,Winter,Sunday,Weekend
2,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,b'tv.jampo.tv',Morning,Winter,Saturday,Weekend
3,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,b'sovsekretno.ru',Morning,Winter,Friday,Workday
4,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,b'marketgid.com',Morning,Winter,Friday,Workday
5,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,b'interfax.ru',Morning,Winter,Friday,Workday


In [163]:
sex_age_csv = '/data/home/valeria.lupanova/project01/sex_age_stat.csv'
sex_age_stat = pd.read_csv(sex_age_csv, sep='\t')

df5_test = pd.merge(df4, sex_age_stat, on=['domain', 'domain'], how='left')

In [164]:
df5_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88736 entries, 0 to 88735
Data columns (total 15 columns):
gender      88736 non-null object
age         88736 non-null object
uid         88736 non-null object
domain      88736 non-null object
day_part    88736 non-null object
season      88736 non-null object
week_day    88736 non-null object
day_type    88736 non-null object
F           81744 non-null float64
M           81744 non-null float64
18-24       81744 non-null float64
25-34       81744 non-null float64
35-44       81744 non-null float64
45-54       81744 non-null float64
>=55        81744 non-null float64
dtypes: float64(7), object(8)
memory usage: 10.8+ MB


In [165]:
df5_test['uid'].unique().shape

(5000,)

In [128]:
for i in ['F', 'M', '18-24', '25-34', '35-44', '45-54', '>=55']:
    df5_test[i].fillna(df5_test[i].mean(), inplace=True)

df5_test.drop(labels='domain',axis=1, inplace=True)

In [129]:
df5_test.head()

,gender,age,uid,day_part,season,week_day,day_type,F,M,18-24,25-34,35-44,45-54,>=55
0,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,Afternoon,Winter,Sunday,Weekend,0.259488,0.740512,0.107679,0.410856,0.238305,0.161959,0.081200
1,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,Morning,Winter,Saturday,Weekend,0.200000,0.800000,0.080000,0.426667,0.400000,0.013333,0.080000
2,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,Morning,Winter,Friday,Workday,0.190789,0.809211,0.026316,0.289474,0.414474,0.217105,0.052632
3,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,Morning,Winter,Friday,Workday,0.348577,0.651423,0.054596,0.295847,0.228651,0.248717,0.172189
4,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,Morning,Winter,Friday,Workday,0.259488,0.740512,0.107679,0.410856,0.238305,0.161959,0.081200


In [130]:
# считываем модели энкодинга
m_le_day_part = "/data/home/valeria.lupanova/project01/le_day_part.pickle"
m_le_season   = "/data/home/valeria.lupanova/project01/le_season  .pickle"
m_le_week_day = "/data/home/valeria.lupanova/project01/le_week_day.pickle"
m_le_day_type = "/data/home/valeria.lupanova/project01/le_day_type.pickle"
m_le_gender   = "/data/home/valeria.lupanova/project01/le_gender  .pickle"
m_le_age      = "/data/home/valeria.lupanova/project01/le_age     .pickle"


le_day_part = pickle.load(open(m_le_day_part, 'rb'))
le_season   = pickle.load(open(m_le_season  , 'rb'))
le_week_day = pickle.load(open(m_le_week_day, 'rb'))
le_day_type = pickle.load(open(m_le_day_type, 'rb'))
le_gender   = pickle.load(open(m_le_gender  , 'rb'))
le_age      = pickle.load(open(m_le_age     , 'rb'))

df5_test['day_part'] = le_day_part.transform(df5_test['day_part'].astype(str))
df5_test['season']   = le_season  .transform(df5_test['season'].astype(str))
df5_test['week_day'] = le_week_day.transform(df5_test['week_day'].astype(str))
df5_test['day_type'] = le_day_type.transform(df5_test['day_type'].astype(str))

#df5_test['gender'] = le_gender.transform(df5_test['gender'].astype(str))
#df5_test['age'] = le_age.transform(df5_test['age'].astype(str))

In [131]:
x = np.array(df5_test.drop(labels=['gender','age','uid'],axis=1))

In [132]:
y = np.array(df5_test[['gender','age']])

In [133]:
#считать модели из файла
model_file_1 = "/data/home/valeria.lupanova/project01/project01_model_gender.pickle"
model_file_2 = "/data/home/valeria.lupanova/project01/project01_model_age.pickle"

p1 = pickle.load(open(model_file_1, 'rb'))
p2 = pickle.load(open(model_file_2, 'rb'))


In [134]:
uid = np.array(df5_test['uid']).reshape(-1,1)

gender = p1.predict(x).reshape(-1,1)

age = p2.predict(x).reshape(-1,1)

uid.shape, gender.shape, age.shape, y.shape

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/anaconda/envs/bd9/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/anaconda/envs/bd9/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/anaconda/envs/bd9/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

((81744, 1), (81744, 1), (81744, 1), (81744, 2))

In [135]:
result = pd.DataFrame(np.hstack([uid,gender,age]), columns=['uid', 'gender','age'])

result['gender'] = result['gender'].astype(int)
result['age'] = result['age'].astype(int)
result['uid'].unique().shape, result.shape

((4949,), (81744, 3))

In [149]:
df4_r = df4[['uid','gender','age']].drop_duplicates(subset=['uid'], keep='first').reset_index(drop=True)
df4_r.head()

,uid,gender,age
0,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,-,-
1,bd7a6f52-45db-49bf-90f2-a3b07a9b7bcd,-,-
2,bd7a7fd9-ab06-42f5-bf0f-1cbb0463004c,-,-
3,bd7c5d7a-0def-41d1-895f-fdb96c56c2d4,-,-
4,bd7e54a2-0215-45cb-a869-9efebf250e38,-,-


In [ ]:
df4_r["uid"].isin(result['uid'])

In [ ]:
df4_r["uid"].isin(result['uid'])

In [144]:
df4_r[df4_r["uid"].isin(result['uid'])]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88736 entries, 0 to 88735
Data columns (total 3 columns):
uid       88736 non-null object
gender    88736 non-null object
age       88736 non-null object
dtypes: object(3)
memory usage: 2.0+ MB


In [150]:
result.head()

,uid,gender,age
0,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,1,1
1,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,1,1
2,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,1,2
3,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,1,1
4,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,1,1


In [151]:
df4_r.head()

,uid,gender,age
0,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,-,-
1,bd7a6f52-45db-49bf-90f2-a3b07a9b7bcd,-,-
2,bd7a7fd9-ab06-42f5-bf0f-1cbb0463004c,-,-
3,bd7c5d7a-0def-41d1-895f-fdb96c56c2d4,-,-
4,bd7e54a2-0215-45cb-a869-9efebf250e38,-,-


In [157]:
len(list(set(df4_r['uid']).intersection(set(result['uid']))))

4949

In [142]:
pd.merge(df4_r, result, on=['uid', 'uid'], how='left').shape

(7745383, 5)

In [136]:
result.head()

,uid,gender,age
0,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,1,1
1,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,1,1
2,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,1,2
3,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,1,1
4,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,1,1


In [65]:
result_fin = result.groupby('uid').agg(lambda x:x.value_counts().index[0]).reset_index(drop=False)
print(result_fin.shape)

(5000, 3)


In [46]:
#res_pd = pd.DataFrame(result_fin[['gender','age']].values == result_fin[['gender_y','age_y']].values)

In [47]:
#res_pd[((res_pd[0] == True) & (res_pd[1] == True))].shape[0]/36138

In [48]:
# сначала вернём обратно лэйблы по энкодингу
result_fin['gender'] = le_gender.inverse_transform(result_fin['gender'])
result_fin['age'] = le_age.inverse_transform(result_fin['age'])

#result_fin['gender_y'] = le_gender.inverse_transform(result_fin['gender_y'])
#result_fin['age_y'] = le_age.inverse_transform(result_fin['age_y'])

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/anaconda/envs/bd9/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [49]:
result_fin.head()

,uid,gender,age
0,0000e7ca-32e6-4bef-bdca-e21c025071ff,F,25-34
1,0000f3cf-6e9a-4eab-92f4-cefdad108c83,M,25-34
2,000381a6-0400-40f8-98c0-93a6c0852d2d,F,25-34
3,00062d89-a3ba-40dd-9a25-212c9f429133,F,25-34
4,0009233e-a0c2-438e-ac36-bd3e1b7dad1e,M,25-34


In [50]:
output = result_fin
output.sort_values(by='uid',axis = 0, ascending = True, inplace = True)
#sys.stdout.write(output.to_json(orient='records'))

In [51]:
output['gender'].value_counts(dropna = False)

M    4049
F     951
Name: gender, dtype: int64

In [52]:
output['age'].value_counts(dropna = False)

25-34    4493
35-44     307
45-54      83
>=55       63
18-24      54
Name: age, dtype: int64

In [74]:
#res_pd = pd.DataFrame(output[['gender','age']].values == output[['gender_y','age_y']].values)

In [53]:
#res_pd[((res_pd[0] == True) & (res_pd[1] == True))].shape[0]/36138

In [ ]:
#t1 = time.clock() - t0
#print("Time elapsed: ", t1) # CPU seconds elapsed (floating point)
#print("SLA: ", t1/output.shape[0])